In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data                         # MNIST data

In [3]:
# load data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
# constants

input_pixels = 784
input_width = 28
input_height = 28
input_channels = 1                # not coloured

n_conv1 = 32                      # number of units in CL 1
n_conv2 = 64                      # number of units in CL 2
stride_conv1 = 1
stride_conv2 = 1

conv1_k = 5                       # size of square filter in CL 1
conv2_k = 5                       # size of square filter in CL 1

max_pool1_k = 2                   # 2x2 pooling
max_pool2_k = 2

n_hidden = 1024                   # number of units in hidden layer
n_out = 10                        # number of units in output layer

input_size_to_hidden = (input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) *n_conv2


In [5]:
# initializing weights and biases

weights = {
    'wc1': tf.Variable(tf.random_normal([conv1_k, conv1_k, input_channels, n_conv1])),
    "wc2": tf.Variable(tf.random_normal([conv2_k, conv2_k, n_conv1, n_conv2])),
    # CL2 is getting number of channels = num of units in CL1
    "wh1": tf.Variable(tf.random_normal([input_size_to_hidden, n_hidden])),
    "wo": tf.Variable(tf.random_normal([n_hidden, n_out]))
    
}

biases = {
    "bc1": tf.Variable(tf.random_normal([n_conv1])),
    "bc2": tf.Variable(tf.random_normal([n_conv2])),
    "bh1": tf.Variable(tf.random_normal([n_hidden])),
    "bo": tf.Variable(tf.random_normal([n_out]))
    
}

In [6]:
def conv(x, weights, bias, strides = 1):
    # apply filter on the images
    out = tf.nn.conv2d(x, weights, padding = 'SAME', strides = [1, strides, strides, 1])
    
    # add bias
    out = tf.nn.bias_add(out, bias)
    
    # apply activation function
    out = tf.nn.relu(out)
    
    return out

In [7]:
def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding = 'SAME', ksize = [1, k, k, 1], strides = [1, k, k, 1])
# [n, w, h, c]

In [23]:
# forward propagation

def cnn(x, weights, biases, keep_prob):
    x = tf.reshape(x, shape= [-1, input_width, input_height, input_channels])
    # reshape() infers the 4th argument (-1) on its by using the remaining parameters passed
    
    conv1 = conv(x, weights['wc1'], biases['bc1'], stride_conv1)
    conv1_pool = maxpooling(conv1, max_pool1_k)
    
    conv2 = conv(conv1_pool, weights['wc2'], biases['bc2'], stride_conv2)
    conv2_pool = maxpooling(conv2, max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool, shape= [-1, input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights['wh1']), biases['bh1'])
    hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    
    # dropout
    hidden_output = tf.nn.dropout(hidden_output_before_dropout, keep_prob)  # requires a keep probability
    
    output = tf.add(tf.matmul(hidden_output, weights['wo']), biases['bo'])
    # no activation function at OL => identity function
    return output

In [28]:
x = tf.placeholder("float", [None, input_pixels])
y = tf.placeholder(tf.int32, [None, n_out])
keep_prob = tf.placeholder("float")
pred = cnn(x, weights, biases, keep_prob)

In [39]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = pred, labels = y))


In [30]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
optimize = optimizer.minimize(cost)

In [31]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [32]:
batch_size = 100
for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimize], feed_dict= {x:batch_x, y:batch_y, keep_prob:0.8})
        total_cost += c
    print(total_cost)

785275.53303051
35457.35568034649
20277.614521607757
15582.253920286894
12436.652538846207
9756.878999612352
8103.262329074439
8876.489596855135
6715.499981495434
6322.816897458956
5794.814815652793
4552.247269296397
5368.716234457323
4436.687722638009
3140.9010596581265
3458.5492105934454
3618.168502415032
3813.8411255661817
2810.8001666362575
2654.0935754806064
2078.341089004947
2393.927128434181
1970.5932851871362
2232.0457742484587
1954.6628133021156


In [33]:
predictions = tf.argmax(pred, 1)
correct_labels = tf.argmax(y, 1)
correct_predictions = tf.equal(predictions, correct_labels)

In [34]:

predictions, correct_predictions = sess.run([predictions, correct_predictions], 
                                      feed_dict = {x: mnist.test.images, y: mnist.test.labels, keep_prob:1.0})
correct = correct_predictions.sum()

In [38]:
print("Score of correctly predicted values : ", (correct/mnist.test.images.shape[0])*100,"%")

Score of correctly predicted values :  98.0 %
